# Budget Optimization

This notebook is to generate optimized budget allocations on top the predictions. From the previous notebooks, there are scenarios for all the combinations that might happen in the designated time frame. For the below code, there are:
- 7 days
- 2 topics
- 3 target groups
- 1 location
- 25 increments for amount spent

When all these numbers are multiplied, we end up up 6300 scenarios in total. For each scenario, our ensembled model predicted a class indication number of conversions. These might be between 0 and 2; 3 and 6; 7 and 12; more than 12. The predictions are made based on the other columns that are listed above. In particular, if we spend more on the marketing campaign which is indicated with the *amount_spent_usd* column, we get more conversions. Straight-forward! But there are daily budget constraints that cannot be exceeded. 

The goal of this notebook is to create the most conversions. That can also be done with spending as much as possible without exceeding the limit. This way it is possible to maximize the conversions without spending huge amounts each day. As the next steps, more constraints can be introduced like weekly or monthly limitations.

## Uploading the Data

In [1]:
import pandas as pd

In [2]:
# Uploading the Facebook Ads Manager from S3 Bucket:
import boto3
from sagemaker import get_execution_role

role = get_execution_role()
bucket='facebookconversionrates/modelling'
data_key = 'predictions_aug.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)

df_pred = pd.read_csv(data_location)
df_pred.drop(columns=['Unnamed: 0'], inplace = True)
df_pred.head()

,year,month,week_num,day_of_week,age,topic,target_grouped,location,amount_spent_usd,pred_rf_gini,...,pred_reg_tree,probs_reg_tree,pred_xgb,probs_xgb,pred_abc_disc,probs_abc_disc,pred_abc_real,probs_abc_real,probs_final,final_prediction
0,2019,8,31,Thurs,18-24,RRE,evergreen,US,10,btw 0 and 2 conversions,...,btw 0 and 2 conversions,"[0.97, 0.03, 0.01, 0.0]",btw 0 and 2 conversions,"[1.0, 0.0, 0.0, 0.0]",btw 0 and 2 conversions,"[0.32, 0.23, 0.23, 0.23]",btw 0 and 2 conversions,"[1.0, 0.0, 0.0, 0.0]","[0.8877777777777778, 0.04666666666666667, 0.03...",btw 0 and 2 conversions
1,2019,8,31,Thurs,18-24,RRE,sqllal,US,10,btw 0 and 2 conversions,...,btw 0 and 2 conversions,"[0.97, 0.03, 0.01, 0.0]",btw 0 and 2 conversions,"[1.0, 0.0, 0.0, 0.0]",btw 0 and 2 conversions,"[0.32, 0.23, 0.23, 0.23]",btw 0 and 2 conversions,"[1.0, 0.0, 0.0, 0.0]","[0.8888888888888888, 0.04666666666666667, 0.03...",btw 0 and 2 conversions
2,2019,8,31,Thurs,18-24,RRE,pagevisitorremarketing,US,10,btw 0 and 2 conversions,...,btw 0 and 2 conversions,"[0.97, 0.03, 0.01, 0.0]",btw 0 and 2 conversions,"[1.0, 0.0, 0.0, 0.0]",btw 0 and 2 conversions,"[0.32, 0.23, 0.23, 0.23]",btw 0 and 2 conversions,"[1.0, 0.0, 0.0, 0.0]","[0.8888888888888888, 0.04666666666666667, 0.03...",btw 0 and 2 conversions
3,2019,8,31,Thurs,18-24,PS,evergreen,US,10,btw 0 and 2 conversions,...,btw 0 and 2 conversions,"[0.97, 0.03, 0.01, 0.0]",btw 0 and 2 conversions,"[1.0, 0.0, 0.0, 0.0]",btw 0 and 2 conversions,"[0.32, 0.23, 0.23, 0.23]",btw 0 and 2 conversions,"[1.0, 0.0, 0.0, 0.0]","[0.8844444444444445, 0.05, 0.03555555555555555...",btw 0 and 2 conversions
4,2019,8,31,Thurs,18-24,PS,entphotographyentrepreneur,US,10,btw 0 and 2 conversions,...,btw 0 and 2 conversions,"[0.97, 0.03, 0.01, 0.0]",btw 0 and 2 conversions,"[1.0, 0.0, 0.0, 0.0]",btw 0 and 2 conversions,"[0.32, 0.23, 0.23, 0.23]",btw 0 and 2 conversions,"[1.0, 0.0, 0.0, 0.0]","[0.8855555555555555, 0.05, 0.03444444444444444...",btw 0 and 2 conversions


In [3]:
df_pred[df_pred['topic'] == 'RRE']['target_grouped'].value_counts()

evergreen                 750
sqllal                    750
pagevisitorremarketing    750
Name: target_grouped, dtype: int64

In [4]:
df_pred[df_pred['topic'] == 'PS']['target_grouped'].value_counts()

evergreen                     750
infographic                   750
entphotographyentrepreneur    750
Name: target_grouped, dtype: int64

In [5]:
df_pred = df_pred[(df_pred['age'] != '18-24') & (df_pred['age'] != '65+')]

In [6]:
df_pred.reset_index(drop=True,inplace=True)

In [7]:
df_pred.shape

(3000, 31)

In [8]:
df_pred

,year,month,week_num,day_of_week,age,topic,target_grouped,location,amount_spent_usd,pred_rf_gini,...,pred_reg_tree,probs_reg_tree,pred_xgb,probs_xgb,pred_abc_disc,probs_abc_disc,pred_abc_real,probs_abc_real,probs_final,final_prediction
0,2019,8,31,Thurs,25-34,RRE,evergreen,US,10,btw 0 and 2 conversions,...,btw 0 and 2 conversions,"[0.97, 0.03, 0.01, 0.0]",btw 0 and 2 conversions,"[1.0, 0.0, 0.0, 0.0]",btw 0 and 2 conversions,"[0.32, 0.23, 0.23, 0.23]",btw 0 and 2 conversions,"[0.99, 0.0, 0.0, 0.0]","[0.8722222222222222, 0.05333333333333333, 0.04...",btw 0 and 2 conversions
1,2019,8,31,Thurs,25-34,RRE,sqllal,US,10,btw 0 and 2 conversions,...,btw 0 and 2 conversions,"[0.97, 0.03, 0.01, 0.0]",btw 0 and 2 conversions,"[1.0, 0.0, 0.0, 0.0]",btw 0 and 2 conversions,"[0.32, 0.23, 0.23, 0.23]",btw 0 and 2 conversions,"[1.0, 0.0, 0.0, 0.0]","[0.8777777777777778, 0.051111111111111114, 0.0...",btw 0 and 2 conversions
2,2019,8,31,Thurs,25-34,RRE,pagevisitorremarketing,US,10,btw 0 and 2 conversions,...,btw 0 and 2 conversions,"[0.97, 0.03, 0.01, 0.0]",btw 0 and 2 conversions,"[1.0, 0.0, 0.0, 0.0]",btw 0 and 2 conversions,"[0.32, 0.23, 0.23, 0.23]",btw 0 and 2 conversions,"[1.0, 0.0, 0.0, 0.0]","[0.8755555555555556, 0.05333333333333333, 0.03...",btw 0 and 2 conversions
3,2019,8,31,Thurs,25-34,PS,evergreen,US,10,btw 0 and 2 conversions,...,btw 0 and 2 conversions,"[0.97, 0.03, 0.01, 0.0]",btw 0 and 2 conversions,"[1.0, 0.0, 0.0, 0.0]",btw 0 and 2 conversions,"[0.32, 0.23, 0.23, 0.23]",btw 0 and 2 conversions,"[0.99, 0.0, 0.01, 0.0]","[0.8677777777777779, 0.05444444444444444, 0.04...",btw 0 and 2 conversions
4,2019,8,31,Thurs,25-34,PS,entphotographyentrepreneur,US,10,btw 0 and 2 conversions,...,btw 0 and 2 conversions,"[0.97, 0.03, 0.01, 0.0]",btw 0 and 2 conversions,"[1.0, 0.0, 0.0, 0.0]",btw 0 and 2 conversions,"[0.32, 0.23, 0.23, 0.23]",btw 0 and 2 conversions,"[1.0, 0.0, 0.0, 0.0]","[0.8677777777777778, 0.05888888888888889, 0.04...",btw 0 and 2 conversions
5,2019,8,31,Thurs,25-34,PS,infographic,US,10,btw 0 and 2 conversions,...,btw 0 and 2 conversions,"[0.97, 0.03, 0.01, 0.0]",btw 0 and 2 conversions,"[1.0, 0.0, 0.0, 0.0]",btw 0 and 2 conversions,"[0.32, 0.23, 0.23, 0.23]",btw 0 and 2 conversions,"[1.0, 0.0, 0.0, 0.0]","[0.8688888888888888, 0.05777777777777778, 0.04...",btw 0 and 2 conversions
6,2019,8,31,Thurs,35-44,RRE,evergreen,US,10,btw 0 and 2 conversions,...,btw 0 and 2 conversions,"[0.97, 0.03, 0.01, 0.0]",btw 0 and 2 conversions,"[1.0, 0.0, 0.0, 0.0]",btw 0 and 2 conversions,"[0.31, 0.23, 0.23, 0.23]",btw 0 and 2 conversions,"[0.97, 0.0, 0.02, 0.0]","[0.8655555555555554, 0.05444444444444444, 0.04...",btw 0 and 2 conversions
7,2019,8,31,Thurs,35-44,RRE,sqllal,US,10,btw 0 and 2 conversions,...,btw 0 and 2 conversions,"[0.97, 0.03, 0.01, 0.0]",btw 0 and 2 conversions,"[1.0, 0.0, 0.0, 0.0]",btw 0 and 2 conversions,"[0.32, 0.23, 0.23, 0.23]",btw 0 and 2 conversions,"[1.0, 0.0, 0.0, 0.0]","[0.8766666666666667, 0.052222222222222225, 0.0...",btw 0 and 2 conversions
8,2019,8,31,Thurs,35-44,RRE,pagevisitorremarketing,US,10,btw 0 and 2 conversions,...,btw 0 and 2 conversions,"[0.97, 0.03, 0.01, 0.0]",btw 0 and 2 conversions,"[1.0, 0.0, 0.0, 0.0]",btw 0 and 2 conversions,"[0.32, 0.23, 0.23, 0.23]",btw 0 and 2 conversions,"[1.0, 0.0, 0.0, 0.0]","[0.8733333333333334, 0.05444444444444444, 0.04...",btw 0 and 2 conversions
9,2019,8,31,Thurs,35-44,PS,evergreen,US,10,btw 0 and 2 conversions,...,btw 0 and 2 conversions,"[0.97, 0.03, 0.01, 0.0]",btw 0 and 2 conversions,"[1.0, 0.0, 0.0, 0.0]",btw 0 and 2 conversions,"[0.31, 0.23, 0.23, 0.23]",btw 0 and 2 conversions,"[0.94, 0.01, 0.05, 0.0]","[0.8499999999999999, 0.05888888888888889, 0.05...",btw 0 and 2 conversions


## Preprocessing I: Converting the Dataframe

In our optimization problem there are:
- days
- age
- location
- topic
- target groups
- amount spent
- final prediction

Since we have daily constraints, we need to take date seperately to the model. On the other hand, we can combine age, location, topic and target grouped. Since there are 6 age groups, 2 topics and 3 target groups, we end up with 36 combinations. The amount spent is another level on top of date and the combination of 36. 

So in general we have 36 combinations of topic, target group and age; 25 for amount spent and 7 for day.

In [9]:
import datetime
import numpy as np

In [10]:
# In order to create the model, we need to merge the year, month, week_num and the day columns.
# That will enable us to understand the date from one column in yyyy-mm-dd format.

# All the strings need to be quantified for that purpose:
val_day = {"Mon": 0, "Tues": 1, "Weds": 2, "Thurs": 3, "Fri": 4, "Sat": 5, "Sun": 6}

# Creating a function that can generate the date in yyyy-mm-dd format:
def getDate(row):
    # Create string format
    str_date = "{0}-W{1}-1".format(row.year, row.week_num - 1)
    # Get the date
    date = datetime.datetime.strptime(
        str_date, "%Y-W%W-%w") + datetime.timedelta(days=val_day[row.day_of_week])
    # Update date field
    row["date"] = date.strftime("%Y-%m-%d")
    return row

# Applying the getDate function to the dataframe:
df_pred = df_pred.apply(getDate, axis=1)

In [11]:
# The model that generated the class predictions is an ensembled model, which means that 
# it includes many models and merge them to find out what the majority predicts. 
# At this stage, we only require the final prediction, the rest might be excluded from the dataframe.
df_opt = df_pred[['date','age','topic','target_grouped','amount_spent_usd','final_prediction']]

In [12]:
# Format of the date columns is changed:
df_opt['date'] = pd.to_datetime(df_opt['date'], format = "%Y-%m-%d")

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [13]:
# For simplicity, the dataframe is sorted in respect to the date and amount spent:
df_opt = df_opt.sort_values(by=['date', 'amount_spent_usd']).reset_index(drop = True)

# First 5 rows of df_opt
df_opt.head()

,date,age,topic,target_grouped,amount_spent_usd,final_prediction
0,2019-08-01,25-34,RRE,evergreen,10,btw 0 and 2 conversions
1,2019-08-01,25-34,RRE,sqllal,10,btw 0 and 2 conversions
2,2019-08-01,25-34,RRE,pagevisitorremarketing,10,btw 0 and 2 conversions
3,2019-08-01,25-34,PS,evergreen,10,btw 0 and 2 conversions
4,2019-08-01,25-34,PS,entphotographyentrepreneur,10,btw 0 and 2 conversions


In [14]:
# To create a checkpoint, the dataframe is saved under the name combinations to both SageMaker and S3.

# Saving the file to SageMaker:
folder_location = '/home/ec2-user/SageMaker/fb_budget_optimization_2019-07-17/'
file_name = 'combinations.csv'
df_opt.to_csv(folder_location + file_name)

In [15]:
# Saving the file to S3:
from io import StringIO
import boto3

bucket='facebookconversionrates' # bucket name
data_key = 'modelling/combinations_pred_aug.csv' # name of the csv file that will be created
csv_buffer = StringIO()
df_opt.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, data_key).put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '7883529152D0C4C5',
  'HostId': 'p9SuECDI6AvL6pBl7hZzvSIoZF24a8dsPMIdQZnXfvnYdAtyl1EbWln1zREuSX7bN/eGBJI4GK4=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'p9SuECDI6AvL6pBl7hZzvSIoZF24a8dsPMIdQZnXfvnYdAtyl1EbWln1zREuSX7bN/eGBJI4GK4=',
   'x-amz-request-id': '7883529152D0C4C5',
   'date': 'Sun, 04 Aug 2019 04:03:44 GMT',
   'etag': '"282c50e428ef1263f5c0b9014730066f"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"282c50e428ef1263f5c0b9014730066f"'}

## Preprocessing II: Quantifying the Columns

After the file is saved, we can start with the second stage of preprocessing. This step includes the iterations that allows strings to be converted to numeric values. The first one is the date. In the previous part, the date is generated in the yyyy-mm-dd format. Now we need to replace the first day with 1 and keep going until the last day.

In [16]:
# Just to check how many days there are in our df_opt.
# We expect 900 for each day since that will make 6300 in total.
values,counts=np.unique(df_opt.date,return_counts=True)
count_dict = dict(sorted(zip(values,counts), key=lambda x: x[0]))
count_dict

{numpy.datetime64('2019-08-01T00:00:00.000000000'): 600,
 numpy.datetime64('2019-08-02T00:00:00.000000000'): 600,
 numpy.datetime64('2019-08-03T00:00:00.000000000'): 600,
 numpy.datetime64('2019-08-04T00:00:00.000000000'): 600,
 numpy.datetime64('2019-08-05T00:00:00.000000000'): 600}

In [17]:
# Since the days need to be quantified, they are replaced by numbers starting from 1.
count_lst = [1] # since the first day will have the number 1

counter = 1 # We need to add on top the first day. If the day changes, counter will be increase by 1.
print(df_opt.date[0]) # First day is printed

for i in range(1,len(df_opt)): 
    if df_opt.date[(i-1)] == df_opt.date[(i)]:
        count_lst.append(counter)
    else:
        counter += 1
        count_lst.append(counter)
        print(df_opt.date[i])
        
# At the end we have a list starting from 1 to 7. 7 because there are 7 days.
# When this list is merged to the dataframe, all days will have their orders.

2019-08-01 00:00:00
2019-08-02 00:00:00
2019-08-03 00:00:00
2019-08-04 00:00:00
2019-08-05 00:00:00


In [18]:
# This above created list is attached to the column called enum_day:
df_opt['enum_day'] = pd.Series(count_lst)

In [19]:
# Check whether we have 7 for the last day:
df_opt.tail()

,date,age,topic,target_grouped,amount_spent_usd,final_prediction,enum_day
2995,2019-08-05,55-64,RRE,sqllal,250,btw 0 and 2 conversions,5
2996,2019-08-05,55-64,RRE,pagevisitorremarketing,250,btw 0 and 2 conversions,5
2997,2019-08-05,55-64,PS,evergreen,250,btw 7 and 12 conversions,5
2998,2019-08-05,55-64,PS,entphotographyentrepreneur,250,btw 3 and 6 conversions,5
2999,2019-08-05,55-64,PS,infographic,250,btw 7 and 12 conversions,5


#### Important: How to quantify the final predictions of the classification model?
Predictions in the df_opt dataframe indicate which range the conversion in that day will fall into. For instance if the prediction is "btw 0 and 2 conversions", it means that the model suggests the conversion will be either 0, 1 or 2. But this value cannot be processed by the model since it is not an integer value. They need to be replaced with an convenient number that resembles each bin. 

In order to do that, I uploaded the CSV file from Facebook Ads Manager which includes all the conversion data from the start of 2018 until mid-July 2019. In this dataset my main concern was to calculate how many conversions occurred each day. This approach enables me to see in how many days there are 0 conversions, 1 conversion or 12 conversions. In other words, I will have the distribution of the conversions. This is very helpful in order to calculate the mean in each group.

In [20]:
# Uploading the Facebook Ads Manager data from S3 Bucket:
from sagemaker import get_execution_role

role = get_execution_role()
bucket='facebookconversionrates/fb_raw_data'
data_key = 'fbadset.xlsx'
data_location = 's3://{}/{}'.format(bucket, data_key)

df = pd.read_excel(data_location, 
                   sheet_name=['sheet1', 'sheet2','sheet3','sheet4','sheet5','sheet6','sheet7','sheet8','sheet9'], 
                   skiprows=None)

In [21]:
#convert dict to df to concat all sheets
dfs = []
for dfname in df.keys():
    temp_df = df[dfname]
    temp_df['Session'] = dfname
    dfs.append(temp_df)
    
df = pd.concat(dfs)

# Sort the dataframe by the date reporting starts:
df.sort_values(by='Reporting starts',ascending=False, inplace = True)

#remove space and concat _column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('_-_', '_')

# Dropping the duplicated rows
df.drop_duplicates(keep=False,inplace=True)

# Replacing the starting time
df['ends'] = df['ends'].replace('Sürekli', 'Ongoing')

# Check the first and the last days:
print("The last day is " + str(max(df['reporting_starts'])))
print("The first day is " + str(min(df['reporting_starts'])))

The last day is 2019-07-19 00:00:00
The first day is 2018-01-01 00:00:00


In [22]:
# To create a checkpoint, a new dataframe is created with the name fb:
fb=df.copy()

# Creating a column to indicate which day of the week that date corresponds
fb['day_of_week']=fb['reporting_starts'].dt.dayofweek
days = {0:'Mon',1:'Tues',2:'Weds',3:'Thurs',4:'Fri',5:'Sat',6:'Sun'}
fb['day_of_week'] = fb['day_of_week'].apply(lambda x: days[x])

# Dividing the starting date into year, month and week number:
fb['year'],fb['month'],fb['week_num'] = fb['reporting_starts'].dt.year, fb['reporting_starts'].dt.month, fb['reporting_starts'].dt.week

# Deleting the unnecessary parts from the campaign name and modifying it:
fb['campaign_name']=fb.campaign_name.str.replace('Facebook','').str.strip().str.replace(' ','')
fb['campaign_name']=fb['campaign_name'].str.replace(' ','').str.strip().apply(lambda x : x[1:] if x.startswith("-") else x)

# Deriving location, topic and type out of campaign name:
fb['location'] = fb.campaign_name.str.split('-').str[0]
fb['topic'] = fb.campaign_name.str.split('-').str[1]
fb['type'] = fb.campaign_name.str.split('-').str[2]

# Deriving target, target_sub and version from ad set name:
fb['target'] = fb.ad_set_name.str.split('-').str[1]
fb['target_sub'] = fb.ad_set_name.str.split('-').str[2]
fb['version']=fb.ad_set_name.str.extract('-([^-]*)$')
fb['target']=fb['target'].str.replace("AEC|CRE|Cloud|Event:Webinar|GEN|INS|MF|MSP|PS|RRE|TH",'').str.replace('-',"")
fb['target']=fb['target'].str.replace("AU|BoostedPost|CA|DE|EMEA|ES|FR|IT|MX|PL|SE|UK|US|ZA",'').str.replace('-',"")
fb['target']=fb['target'].str.replace("AU|BoostedPost|CA|DE|EMEA|ES|FR|IT|MX|PL|SE|UK|US|ZA|LAL|Latin America",'').str.replace('-',"")

# Generating the target group:
import re
fb['target_grouped'] = fb['target'].astype(str) + fb['target_sub'].astype(str)
fb['target_grouped'].replace(regex=True, inplace=True, to_replace=r'[^A-Za-z0-9]+', value='')
fb.target_grouped=fb.target_grouped.str.strip().str.lower().str.replace(' ', '')
fb['target_grouped']=fb.target_grouped.str.replace('openedleadformslal','openedleadformlal')
fb['target_grouped']=fb.target_grouped.str.replace('MQLSQLPS','MQLSQL')

# Revising the topic:
fb['topic']=fb.topic.str.replace('Clould3.0','Cloud3.0')
fb['topic']=fb.topic.str.replace('CarribeanIslands','CaribbeanIslands')
fb['topic']=fb.topic.str.replace('AECArchitects','AEC')
fb['topic']=fb.topic.str.replace('CAWebsiteRetargeting','RetargetingWebsite')
fb['topic']=fb.topic.str.replace('CAWebsiteRetargeting','RetargetingWebsite')
fb['topic']=fb.topic.str.replace('WebsiteRetargeting','RetargetingWebsite')
fb['target']=fb.target.str.replace('Re','Re-Engagement')
fb['target']=fb.target.str.replace('&','')

# Usually it is not recommended to run so many code in one cell but since they are already done in
# detailed in another notebook, I didn't want to spend extra space.

In [23]:
# Querying the dataframe in respect to the below ages:
age=['18-24','25-34', '35-44', '45-54', '55-64','65+']
fb=fb.query('age in @age')

In [24]:
# Fill all blanks with novalue if the column dtype is object
fb = fb.apply(lambda x: x.fillna(0.0) if x.dtype.kind in 'biufc' else x.fillna('novalue'))

In [25]:
fb=fb.fillna(0.0)      #if the blanks are nan will need this line first

In [26]:
# Grouping by the campaign names:
fb[(fb['location']=='US')&(fb['year']==2019) &(fb['topic']=='RRE')
   ].groupby(['campaign_name','ad_set_name','target_grouped']).size()

campaign_name           ad_set_name                                            target_grouped        
US-RRE-LeadSubmissions  US - RRE - Agents & Brokers - LAL - 10.17              agentsbrokers             128
                        US - RRE - Camera Sale LAL - Evergreen - 4.24          camerasalelal             108
                        US - RRE - Evergreen - Agents & Brokers - LAL - 4.10   evergreen                  72
                        US - RRE - Evergreen - Agents & Brokers - LAL - 4.19   evergreen                 128
                        US - RRE - Evergreen - Video Remarketing & LAL - 4.18  evergreen                 282
                        US - RRE - Evergreen - Video Remarketing & LAL - 4.4   evergreen                  92
                        US - RRE - Evergreen - Video Remarketing & LAL - 5.21  evergreen                   8
                        US - RRE - Feb Promo - Agents & Brokers - LAL - 2.1    febpromo                  112
                        US

In [27]:
# Now the main goal is to create the conversions from the dataframe. 
# There is a column that includes the conversion numbers of each day.
# After saving that as float, we have the numbers:
fb['conversion']= fb[['lead_page_contains_thank-you']].astype(float)

In [28]:
# Creating a dataframe to check each conversion number and their frequencies
conversions = pd.DataFrame(fb.conversion.value_counts()).reset_index()
conversions = conversions.rename(columns = {'conversion':'occurance',
                                            'index':'conversion'})

In [29]:
# Creating the labels to group them easily:
conv_lst = []
for i in range(len(conversions)):
    if ((conversions.conversion[i] >= 0) & (conversions.conversion[i] <= 2)):
        conv_lst.append('btw 0 and 2 conversions')
    elif ((conversions.conversion[i] >= 3) & (conversions.conversion[i] <= 6)):
        conv_lst.append('btw 3 and 6 conversions')
    elif ((conversions.conversion[i] >= 7) & (conversions.conversion[i] <= 12)):
        conv_lst.append('btw 7 and 12 conversions')
    elif (conversions.conversion[i] > 12):
        conv_lst.append('more than 12 conversions')
    else:
        print('Error')
        break

In [30]:
# Assigning the list to the new columns called label:      
conversions['label'] = conv_lst     

# Creating another column as the multiplication of index and its occurance:
conversions['conversion*occurance'] = conversions['conversion'] * conversions['occurance']

# Checking the dataframe:
conversions

,conversion,occurance,label,conversion*occurance
0,0.0,69197,btw 0 and 2 conversions,0.0
1,1.0,11885,btw 0 and 2 conversions,11885.0
2,2.0,6657,btw 0 and 2 conversions,13314.0
3,3.0,3304,btw 3 and 6 conversions,9912.0
4,4.0,1984,btw 3 and 6 conversions,7936.0
5,5.0,1200,btw 3 and 6 conversions,6000.0
6,6.0,722,btw 3 and 6 conversions,4332.0
7,7.0,537,btw 7 and 12 conversions,3759.0
8,8.0,349,btw 7 and 12 conversions,2792.0
9,9.0,301,btw 7 and 12 conversions,2709.0


In [31]:
# Grouping by the sum for each label
conversions[['label','conversion*occurance']].groupby(['label']).sum()

,conversion*occurance
label,
btw 0 and 2 conversions,25199.0
btw 3 and 6 conversions,28180.0
btw 7 and 12 conversions,15046.0
more than 12 conversions,5828.0


In [32]:
# Grouping by the sum of each label 
conversions[['occurance','label']].groupby(['label']).sum()

,occurance
label,
btw 0 and 2 conversions,87739
btw 3 and 6 conversions,7210
btw 7 and 12 conversions,1722
more than 12 conversions,354


In [33]:
# weighted_df is created to understand the means of each bin. 
# At the end of this cell, we are able to check the averages of each bin dependind on the frequency of the conversions.
weighted_df = pd.merge(conversions[['label','conversion*occurance']].groupby(['label']).sum(),
                       conversions[['occurance','label']].groupby(['label']).sum(), 
                       on = 'label')

weighted_df['avg'] = weighted_df['conversion*occurance'] / weighted_df['occurance']
weighted_df

,conversion*occurance,occurance,avg
label,,,
btw 0 and 2 conversions,25199.0,87739,0.287204
btw 3 and 6 conversions,28180.0,7210,3.908460
btw 7 and 12 conversions,15046.0,1722,8.737515
more than 12 conversions,5828.0,354,16.463277


In [34]:
weighted_df['avg'][0] = -1

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [35]:
weighted_df['avg'].tolist()

[-1.0, 3.9084604715672677, 8.737514518002323, 16.463276836158194]

In [36]:
sorted(list(weighted_df.index))

['btw 0 and 2 conversions',
 'btw 3 and 6 conversions',
 'btw 7 and 12 conversions',
 'more than 12 conversions']

In [37]:
# After calculating the means for each bin, a dictionary is created for the further stage of optimization:
labels = dict(zip(sorted(list(weighted_df.index)), weighted_df['avg'].tolist()))
labels

{'btw 0 and 2 conversions': -1.0,
 'btw 3 and 6 conversions': 3.9084604715672677,
 'btw 7 and 12 conversions': 8.737514518002323,
 'more than 12 conversions': 16.463276836158194}

## Building the Optimization Problem

We can build the optimization problem with the following equations:

**Objective Function:**
$$ max \sum_{i=1}^{d} \sum_{j=1}^{c} \sum_{k=1}^{a}{\text{prediction}_{i,j,k} * x_{i,j,k}} $$ 

**Constraints:**

Daily Budget: 
$$ \sum_{j=1}^{c} \sum_{k=1}^{a}{10*k*x_{d,j,k}} \leq \text{daily budget}\text{..............} \forall{d} $$ 
Weekly Budget:

$$ \sum_{i=1}^{d} \sum_{j=1}^{c} \sum_{k=1}^{a}{10*k*x_{i,j,k}} \leq \text{weekly budget} $$

Non-negativity:

$$ x_{i,j,k} \geq 0 $$

where
- *a* is all the budget opportunities by increments of 10
- *d* is all the days user entered
- *c* is all the combinations of age, topic and target_grouped

In order to create the variable `c`, we need to combine all the age, topic and target grouped columns. Since there are 36 combinations of those three, I assigned each one of them a number from 1 to 36. This way, it is easier to introduce them to the model.

In [38]:
df_opt['combination'] = df_opt['age'] + '_' + df_opt['topic'] + '_' + df_opt['target_grouped']

In [39]:
keys = df_opt.combination.value_counts().index.tolist() # creating a list of all combinations of age, topic and group
values = list(range(1,(len(df_opt.combination.value_counts().index.tolist())+1))) # range from 1 until the length of keys
combin_dict = dict(zip(keys,values)) # dictionary that matches the combinations with numbers.

df_opt['combination'] = df_opt['combination'].apply(lambda x: combin_dict[x]) # combinations are replaced by their number

In [40]:
# Sorting the dataframe:
df_opt = df_opt.sort_values(['enum_day','combination','amount_spent_usd']).reset_index(drop=True)

In [41]:
# Created the numeric value of the final prediction depending on the bin means from the distribution:
df_opt['enum_pred'] = df_opt['final_prediction'].apply(lambda x: labels[x])

In [42]:
df_opt

,date,age,topic,target_grouped,amount_spent_usd,final_prediction,enum_day,combination,enum_pred
0,2019-08-01,35-44,PS,entphotographyentrepreneur,10,btw 0 and 2 conversions,1,1,-1.00000
1,2019-08-01,35-44,PS,entphotographyentrepreneur,20,btw 0 and 2 conversions,1,1,-1.00000
2,2019-08-01,35-44,PS,entphotographyentrepreneur,30,btw 0 and 2 conversions,1,1,-1.00000
3,2019-08-01,35-44,PS,entphotographyentrepreneur,40,btw 0 and 2 conversions,1,1,-1.00000
4,2019-08-01,35-44,PS,entphotographyentrepreneur,50,btw 0 and 2 conversions,1,1,-1.00000
5,2019-08-01,35-44,PS,entphotographyentrepreneur,60,btw 0 and 2 conversions,1,1,-1.00000
6,2019-08-01,35-44,PS,entphotographyentrepreneur,70,btw 0 and 2 conversions,1,1,-1.00000
7,2019-08-01,35-44,PS,entphotographyentrepreneur,80,btw 0 and 2 conversions,1,1,-1.00000
8,2019-08-01,35-44,PS,entphotographyentrepreneur,90,btw 0 and 2 conversions,1,1,-1.00000
9,2019-08-01,35-44,PS,entphotographyentrepreneur,100,btw 0 and 2 conversions,1,1,-1.00000


In [51]:
d = 5 # 5 days
c = 24 # 2 topics * 4 age groups * 3 target groups = 24 combinations
a = 60 # 60 amount spent increments

# Ranges for all the variables
set_I = range(1, d+1) 
set_J = range(1, c+1)
set_K = range(1, a+1)

# Since the decision variable will have three components (day, combination and amount spent), we need to combine them with 
# final predictions. This way, the predictions can be linked with the scenarios.
enum_pred = df_opt['enum_pred']
preds = {(i,j,k): 0 for i in set_I for j in set_J for k in set_K}
dict_opt = dict(zip(list(preds.keys()),enum_pred))

budget_d = 3000
budget_w = 30000

In [52]:
# In case you get an error from the below code, run the pip install command.
# !pip install pulp

In [53]:
# Initiate the model
import pulp as plp
opt_model = plp.LpProblem(name="MIP Model")

In [54]:
# Since x is Binary
x_vars  = {(i,j,k):
plp.LpVariable(cat=plp.LpBinary, name="x_{0}_{1}_{2}".format(i,j,k)) 
for i in set_I for j in set_J for k in set_K}

In [55]:
x_vars

{(1, 1, 1): x_1_1_1,
 (1, 1, 2): x_1_1_2,
 (1, 1, 3): x_1_1_3,
 (1, 1, 4): x_1_1_4,
 (1, 1, 5): x_1_1_5,
 (1, 1, 6): x_1_1_6,
 (1, 1, 7): x_1_1_7,
 (1, 1, 8): x_1_1_8,
 (1, 1, 9): x_1_1_9,
 (1, 1, 10): x_1_1_10,
 (1, 1, 11): x_1_1_11,
 (1, 1, 12): x_1_1_12,
 (1, 1, 13): x_1_1_13,
 (1, 1, 14): x_1_1_14,
 (1, 1, 15): x_1_1_15,
 (1, 1, 16): x_1_1_16,
 (1, 1, 17): x_1_1_17,
 (1, 1, 18): x_1_1_18,
 (1, 1, 19): x_1_1_19,
 (1, 1, 20): x_1_1_20,
 (1, 1, 21): x_1_1_21,
 (1, 1, 22): x_1_1_22,
 (1, 1, 23): x_1_1_23,
 (1, 1, 24): x_1_1_24,
 (1, 1, 25): x_1_1_25,
 (1, 1, 26): x_1_1_26,
 (1, 1, 27): x_1_1_27,
 (1, 1, 28): x_1_1_28,
 (1, 1, 29): x_1_1_29,
 (1, 1, 30): x_1_1_30,
 (1, 1, 31): x_1_1_31,
 (1, 1, 32): x_1_1_32,
 (1, 1, 33): x_1_1_33,
 (1, 1, 34): x_1_1_34,
 (1, 1, 35): x_1_1_35,
 (1, 1, 36): x_1_1_36,
 (1, 1, 37): x_1_1_37,
 (1, 1, 38): x_1_1_38,
 (1, 1, 39): x_1_1_39,
 (1, 1, 40): x_1_1_40,
 (1, 1, 41): x_1_1_41,
 (1, 1, 42): x_1_1_42,
 (1, 1, 43): x_1_1_43,
 (1, 1, 44): x_1_1_44,
 (1, 1

In [56]:
# Less than equal constraints for daily budget
constraints = {i : 
plp.LpConstraint(
             e=sum(10 * k * x_vars[i,j,k] for j in set_J for k in set_K),
             sense=plp.LpConstraintLE,
             rhs=budget_d,
             name="constraint_{0}".format(i))
       for i in set_I}

In [57]:
constraints

{1: 10*x_1_10_1 + 100*x_1_10_10 + 110*x_1_10_11 + 120*x_1_10_12 + 130*x_1_10_13 + 140*x_1_10_14 + 150*x_1_10_15 + 160*x_1_10_16 + 170*x_1_10_17 + 180*x_1_10_18 + 190*x_1_10_19 + 20*x_1_10_2 + 200*x_1_10_20 + 210*x_1_10_21 + 220*x_1_10_22 + 230*x_1_10_23 + 240*x_1_10_24 + 250*x_1_10_25 + 260*x_1_10_26 + 270*x_1_10_27 + 280*x_1_10_28 + 290*x_1_10_29 + 30*x_1_10_3 + 300*x_1_10_30 + 310*x_1_10_31 + 320*x_1_10_32 + 330*x_1_10_33 + 340*x_1_10_34 + 350*x_1_10_35 + 360*x_1_10_36 + 370*x_1_10_37 + 380*x_1_10_38 + 390*x_1_10_39 + 40*x_1_10_4 + 400*x_1_10_40 + 410*x_1_10_41 + 420*x_1_10_42 + 430*x_1_10_43 + 440*x_1_10_44 + 450*x_1_10_45 + 460*x_1_10_46 + 470*x_1_10_47 + 480*x_1_10_48 + 490*x_1_10_49 + 50*x_1_10_5 + 500*x_1_10_50 + 510*x_1_10_51 + 520*x_1_10_52 + 530*x_1_10_53 + 540*x_1_10_54 + 550*x_1_10_55 + 560*x_1_10_56 + 570*x_1_10_57 + 580*x_1_10_58 + 590*x_1_10_59 + 60*x_1_10_6 + 600*x_1_10_60 + 70*x_1_10_7 + 80*x_1_10_8 + 90*x_1_10_9 + 10*x_1_11_1 + 100*x_1_11_10 + 110*x_1_11_11 + 120*x_1_

In [58]:
# For the objective function that will be maximized:
objective = plp.lpSum(x_vars[i,j,k] * dict_opt[i,j,k] 
                    for i in set_I 
                    for j in set_J
                    for k in set_K)

KeyError: (3, 3, 1)

In [ ]:
objective

In [ ]:
# Adding the constraints to the model
opt_model.constraints = constraints

In [ ]:
# For maximization
opt_model.sense = plp.LpMaximize

In [ ]:
# Adding the objective to the model
opt_model.setObjective(objective)

In [ ]:
# solving with CBC
opt_model.solve()

In [ ]:
opt_model

In [ ]:
# The best way to see the result is to reflect them on a dataframe
opt_df = pd.DataFrame.from_dict(x_vars, orient="index", columns = ["variable_object"])

# Arranging the index
opt_df.index =  pd.MultiIndex.from_tuples(opt_df.index, names=["column_i", "column_j", "column_k"])
opt_df.reset_index(inplace=True)

# PuLP
opt_df["solution_value"] =   opt_df["variable_object"].apply(lambda item: item.varValue)
opt_df.drop(columns=["variable_object"], inplace=True)

In [ ]:
# Checking how many of the campaigns are suggested:
opt_df.solution_value.value_counts()

In [ ]:
opt_df.head()

In [ ]:
# Changing the column names
opt_df = opt_df.rename(columns = {'column_i':'day',
                         'column_j':'combination',
                         'column_k':'amount_spent_usd',
                         'solution_value':'solution'})

In [ ]:
# In order to understand which number corresponds to what combination, a dictionary created.
# This way it will be enabled to convert combination numbers into combination contents.
combin_dict_rvr = dict(zip(list(combin_dict.values()),list(combin_dict.keys())))
opt_df['combination'] = opt_df['combination'].apply(lambda x: combin_dict_rvr[x])

In [ ]:
opt_df.combination[:8]

In [ ]:
# Split the combinations just like we combined them in the first iterations: 
opt_df['age'] = opt_df['combination'].str.split('_').str[0]
opt_df['topic'] = opt_df['combination'].str.split('_').str[1]
opt_df['target_grouped'] = opt_df['combination'].str.split('_').str[2]

In [ ]:
# Drop the combination column:
opt_df.drop(columns=['combination'], inplace = True)

In [ ]:
# Multiply amount spent with 10 since it wasn't
opt_df['amount_spent_usd'] = opt_df['amount_spent_usd']*10

In [ ]:
opt_df.head()

In [ ]:
# The suggested campaigns in the first day:
opt_df[opt_df['solution'] == 1][opt_df['day'] == 1]

In [ ]:
# Let's convert the day column to date format yyyy-mm-dd:
import datetime

start = datetime.datetime.strptime("2019-08-01", "%Y-%m-%d") # need to be changed with the user input!
end = datetime.datetime.strptime("2019-08-08", "%Y-%m-%d") # need to be changed with the user input!
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

date_lst = []
for date in date_generated:
    date_lst.append(date.strftime("%Y-%m-%d"))
date_rng = list(range(1,8))

In [ ]:
# Creating the date column based on the day column
dict1 = dict(zip(date_rng, date_lst))
opt_df['date'] = opt_df['day'].apply(lambda x: dict1[x])

In [ ]:
# Converting to the datetime format
opt_df['date'] = pd.to_datetime(opt_df['date'], format = "%Y-%m-%d", errors='coerce')

In [ ]:
# Merging df_opt and opt_df to see which combinations are recommended
opt_df_merged = df_opt.merge(opt_df[['amount_spent_usd','age','topic','target_grouped','date','solution']], 
                             on=['amount_spent_usd','age','topic','target_grouped','date'])

In [ ]:
# Excluding the unnecessary columns:
opt_df_merged = opt_df_merged[['date','age','topic','target_grouped','amount_spent_usd','final_prediction','solution']]

In [ ]:
len(opt_df_merged)

In [ ]:
# Distribution of compaigns over labels:
opt_df_merged[opt_df_merged['solution'] == 1].final_prediction.value_counts()

In [ ]:
# How much did we spend each day?
opt_df_merged[opt_df_merged['solution'] == 1].groupby(['date']).sum()

Above chart proves that our optimization model works. The objective was to fit as much marketing campaigns as possible to the daily budget so that number of conversions can be maximized. The graph proves that it functions perfectly by filling the quota each day.

In [ ]:
# We also need to create a column indicating which day that date corresponds to:
opt_df_merged['day_of_week'] = opt_df_merged['date'].dt.dayofweek
days = {0:'Mon',1:'Tues',2:'Weds',3:'Thurs',4:'Fri',5:'Sat',6:'Sun'}
opt_df_merged['day_of_week'] = opt_df_merged['day_of_week'].apply(lambda x: days[x])

In [ ]:
# Rearranging the column order:
opt_df_merged = opt_df_merged[['date', 'day_of_week', 'age', 'topic', 'target_grouped', 'amount_spent_usd', 'final_prediction', 'solution']]

In [ ]:
# Dataframe that includes the weight for each label:
value_df = pd.DataFrame(list(labels.keys()), columns = ['index'])
value_df['weight'] = list(labels.values())
value_df

In [ ]:
# Dataframe that shows the amount of 0-2, 3-6, 7-12 and 12+ conversions
df1 = pd.DataFrame(opt_df_merged[opt_df_merged['solution'] == 1].final_prediction.value_counts()).reset_index().sort_values(['index'])
df1

In [ ]:
# Merging both:
evaluate_df = pd.merge(value_df, df1, how = 'left', on = ['index']).fillna(0)
evaluate_df

In [ ]:
'The expected amount of conversions for the designated period is: {}.'.format(int(sum(evaluate_df['weight'] * evaluate_df['final_prediction'])))

In [ ]:
opt_df_merged.to_csv('/home/ec2-user/SageMaker/fb_budget_optimization_2019-07-17/optimized_aug.csv')

In [ ]:
# Saving the file to S3:
from io import StringIO
import boto3

bucket='facebookconversionrates' # bucket name
data_key = 'modelling/optimized_aug.csv' # name of the csv file that will be created
csv_buffer = StringIO()
opt_df_merged.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, data_key).put(Body=csv_buffer.getvalue())

## For PS

In [ ]:
df_opt_ps = df_opt[df_opt['topic'] == 'PS'].reset_index(drop=True)

In [ ]:
df_opt_ps['combination'] = df_opt_ps['age'] + '_' + df_opt_ps['topic'] + '_' + df_opt_ps['target_grouped']

In [ ]:
keys = df_opt_ps.combination.value_counts().index.tolist() # creating a list of all combinations of age, topic and group
values = list(range(1,(len(df_opt_ps.combination.value_counts().index.tolist())+1))) # range from 1 until the length of keys
combin_dict = dict(zip(keys,values)) # dictionary that matches the combinations with numbers.

df_opt_ps['combination'] = df_opt_ps['combination'].apply(lambda x: combin_dict[x]) # combinations are replaced by their number

In [ ]:
# Sorting the dataframe:
df_opt_ps = df_opt_ps.sort_values(['enum_day','combination','amount_spent_usd']).reset_index(drop=True)

In [ ]:
# Created the numeric value of the final prediction depending on the bin means from the distribution:
df_opt_ps['enum_pred'] = df_opt_ps['final_prediction'].apply(lambda x: labels[x])

In [ ]:
df_opt_ps

In [ ]:
d = 7 # 7 days
c = 12 # 1 topics * 4 age groups * 3 target groups = 12 combinations
a = 25 # 25 amount spent increments

# Ranges for all the variables
set_I = range(1, d+1) 
set_J = range(1, c+1)
set_K = range(1, a+1)

# Since the decision variable will have three components (day, combination and amount spent), we need to combine them with 
# final predictions. This way, the predictions can be linked with the scenarios.
enum_pred = df_opt_ps['enum_pred']
preds = {(i,j,k): 0 for i in set_I for j in set_J for k in set_K}
dict_opt = dict(zip(list(preds.keys()),enum_pred))

budget_d = 2000
budget_w = 30000

In [ ]:
# In case you get an error from the below code, run the pip install command.
# !pip install pulp

In [ ]:
# Initiate the model
import pulp as plp
opt_model = plp.LpProblem(name="MIP Model")

In [ ]:
# Since x is Binary
x_vars  = {(i,j,k):
plp.LpVariable(cat=plp.LpBinary, name="x_{0}_{1}_{2}".format(i,j,k)) 
for i in set_I for j in set_J for k in set_K}

In [ ]:
x_vars

In [ ]:
# Less than equal constraints for daily budget
constraints = {i : 
plp.LpConstraint(
             e=sum(10 * k * x_vars[i,j,k] for j in set_J for k in set_K),
             sense=plp.LpConstraintLE,
             rhs=budget_d,
             name="constraint_{0}".format(i))
       for i in set_I}

In [ ]:
constraints

In [ ]:
# For the objective function that will be maximized:
objective = plp.lpSum(x_vars[i,j,k] * dict_opt[i,j,k] 
                    for i in set_I 
                    for j in set_J
                    for k in set_K)

In [ ]:
objective

In [ ]:
# Adding the constraints to the model
opt_model.constraints = constraints

In [ ]:
# For maximization
opt_model.sense = plp.LpMaximize

In [ ]:
# Adding the objective to the model
opt_model.setObjective(objective)

In [ ]:
# solving with CBC
opt_model.solve()

In [ ]:
opt_model

In [ ]:
# The best way to see the result is to reflect them on a dataframe
opt_df_ps = pd.DataFrame.from_dict(x_vars, orient="index", columns = ["variable_object"])

# Arranging the index
opt_df_ps.index =  pd.MultiIndex.from_tuples(opt_df_ps.index, names=["column_i", "column_j", "column_k"])
opt_df_ps.reset_index(inplace=True)

# PuLP
opt_df_ps["solution_value"] =   opt_df_ps["variable_object"].apply(lambda item: item.varValue)
opt_df_ps.drop(columns=["variable_object"], inplace=True)

In [ ]:
# Checking how many of the campaigns are suggested:
opt_df_ps.solution_value.value_counts()

In [ ]:
opt_df_ps.head()

In [ ]:
# Changing the column names
opt_df_ps = opt_df_ps.rename(columns = {'column_i':'day',
                         'column_j':'combination',
                         'column_k':'amount_spent_usd',
                         'solution_value':'solution'})

In [ ]:
# In order to understand which number corresponds to what combination, a dictionary created.
# This way it will be enabled to convert combination numbers into combination contents.
combin_dict_rvr = dict(zip(list(combin_dict.values()),list(combin_dict.keys())))
opt_df_ps['combination'] = opt_df_ps['combination'].apply(lambda x: combin_dict_rvr[x])

In [ ]:
opt_df_ps.combination[:8]

In [ ]:
# Split the combinations just like we combined them in the first iterations: 
opt_df_ps['age'] = opt_df_ps['combination'].str.split('_').str[0]
opt_df_ps['topic'] = opt_df_ps['combination'].str.split('_').str[1]
opt_df_ps['target_grouped'] = opt_df_ps['combination'].str.split('_').str[2]

In [ ]:
# Drop the combination column:
opt_df_ps.drop(columns=['combination'], inplace = True)

In [ ]:
# Multiply amount spent with 10 since it wasn't
opt_df_ps['amount_spent_usd'] = opt_df_ps['amount_spent_usd']*10

In [ ]:
opt_df_ps.head()

In [ ]:
# The suggested campaigns in the first day:
opt_df_ps[opt_df_ps['solution'] == 1][opt_df_ps['day'] == 1]

In [ ]:
# Let's convert the day column to date format yyyy-mm-dd:
import datetime

start = datetime.datetime.strptime("2019-08-01", "%Y-%m-%d") # need to be changed with the user input!
end = datetime.datetime.strptime("2019-08-08", "%Y-%m-%d") # need to be changed with the user input!
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

date_lst = []
for date in date_generated:
    date_lst.append(date.strftime("%Y-%m-%d"))
date_rng = list(range(1,8))

In [ ]:
# Creating the date column based on the day column
dict1 = dict(zip(date_rng, date_lst))
opt_df_ps['date'] = opt_df_ps['day'].apply(lambda x: dict1[x])

In [ ]:
# Converting to the datetime format
opt_df_ps['date'] = pd.to_datetime(opt_df_ps['date'], format = "%Y-%m-%d", errors='coerce')

In [ ]:
# Merging df_opt and opt_df to see which combinations are recommended
opt_df_ps_merged = df_opt_ps.merge(opt_df_ps[['amount_spent_usd','age','topic','target_grouped','date','solution']], 
                             on=['amount_spent_usd','age','topic','target_grouped','date'])

In [ ]:
# Excluding the unnecessary columns:
opt_df_ps_merged = opt_df_ps_merged[['date','age','topic','target_grouped','amount_spent_usd','final_prediction','solution']]

In [ ]:
len(opt_df_ps_merged)

In [ ]:
# Distribution of compaigns over labels:
opt_df_ps_merged[opt_df_ps_merged['solution'] == 1].final_prediction.value_counts()

In [ ]:
# How much did we spend each day?
opt_df_ps_merged[opt_df_ps_merged['solution'] == 1].groupby(['date']).sum()

Above chart proves that our optimization model works. The objective was to fit as much marketing campaigns as possible to the daily budget so that number of conversions can be maximized. The graph proves that it functions perfectly by filling the quota each day.

In [ ]:
# We also need to create a column indicating which day that date corresponds to:
opt_df_ps_merged['day_of_week'] = opt_df_ps_merged['date'].dt.dayofweek
days = {0:'Mon',1:'Tues',2:'Weds',3:'Thurs',4:'Fri',5:'Sat',6:'Sun'}
opt_df_ps_merged['day_of_week'] = opt_df_ps_merged['day_of_week'].apply(lambda x: days[x])

In [ ]:
# Rearranging the column order:
opt_df_ps_merged = opt_df_ps_merged[['date', 'day_of_week', 'age', 'topic', 'target_grouped', 'amount_spent_usd', 'final_prediction', 'solution']]

In [ ]:
# Dataframe that includes the weight for each label:
value_df = pd.DataFrame(list(labels.keys()), columns = ['index'])
value_df['weight'] = list(labels.values())
value_df

In [ ]:
# Dataframe that shows the amount of 0-2, 3-6, 7-12 and 12+ conversions
df1 = pd.DataFrame(opt_df_ps_merged[opt_df_ps_merged['solution'] == 1].final_prediction.value_counts()).reset_index().sort_values(['index'])
df1

In [ ]:
# Merging both:
evaluate_df = pd.merge(value_df, df1, how = 'left', on = ['index']).fillna(0)
evaluate_df

In [ ]:
'The expected amount of conversions for the designated period is: {}.'.format(int(sum(evaluate_df['weight'] * evaluate_df['final_prediction'])))

In [ ]:
opt_df_ps_merged.to_csv('/home/ec2-user/SageMaker/fb_budget_optimization_2019-07-17/optimized_aug_ps.csv')

In [ ]:
# Saving the file to S3:
from io import StringIO
import boto3

bucket='facebookconversionrates' # bucket name
data_key = 'modelling/optimized_aug_ps.csv' # name of the csv file that will be created
csv_buffer = StringIO()
opt_df_ps_merged.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, data_key).put(Body=csv_buffer.getvalue())

In [ ]:
opt_df_ps.target_grouped.value_counts()

## For RRE

In [ ]:
df_opt_rre.final_prediction.value_counts()

In [ ]:
df_opt_rre[df_opt_rre['final_prediction'] == 'btw 3 and 6 conversions']

In [ ]:
df_opt_rre = df_opt[df_opt['topic'] == 'RRE'].reset_index(drop=True)

In [ ]:
df_opt_rre['combination'] = df_opt_rre['age'] + '_' + df_opt_rre['topic'] + '_' + df_opt_rre['target_grouped']

In [ ]:
keys = df_opt_rre.combination.value_counts().index.tolist() # creating a list of all combinations of age, topic and group
values = list(range(1,(len(df_opt_rre.combination.value_counts().index.tolist())+1))) # range from 1 until the length of keys
combin_dict = dict(zip(keys,values)) # dictionary that matches the combinations with numbers.

df_opt_rre['combination'] = df_opt_rre['combination'].apply(lambda x: combin_dict[x]) # combinations are replaced by their number

In [ ]:
# Sorting the dataframe:
df_opt_rre = df_opt_rre.sort_values(['enum_day','combination','amount_spent_usd']).reset_index(drop=True)

In [ ]:
# Created the numeric value of the final prediction depending on the bin means from the distribution:
df_opt_rre['enum_pred'] = df_opt_rre['final_prediction'].apply(lambda x: labels[x])

In [ ]:
df_opt_rre

In [ ]:
d = 7 # 7 days
c = 12 # 1 topics * 4 age groups * 3 target groups = 12 combinations
a = 25 # 25 amount spent increments

# Ranges for all the variables
set_I = range(1, d+1) 
set_J = range(1, c+1)
set_K = range(1, a+1)

# Since the decision variable will have three components (day, combination and amount spent), we need to combine them with 
# final predictions. This way, the predictions can be linked with the scenarios.
enum_pred = df_opt_rre['enum_pred']
preds = {(i,j,k): 0 for i in set_I for j in set_J for k in set_K}
dict_opt = dict(zip(list(preds.keys()),enum_pred))

budget_d = 600
budget_w = 30000

In [ ]:
# In case you get an error from the below code, run the pip install command.
# !pip install pulp

In [ ]:
# Initiate the model
import pulp as plp
opt_model = plp.LpProblem(name="MIP Model")

In [ ]:
# Since x is Binary
x_vars  = {(i,j,k):
plp.LpVariable(cat=plp.LpBinary, name="x_{0}_{1}_{2}".format(i,j,k)) 
for i in set_I for j in set_J for k in set_K}

In [ ]:
x_vars

In [ ]:
# Less than equal constraints for daily budget
constraints = {i : 
plp.LpConstraint(
             e=sum(10 * k * x_vars[i,j,k] for j in set_J for k in set_K),
             sense=plp.LpConstraintLE,
             rhs=budget_d,
             name="constraint_{0}".format(i))
       for i in set_I}

In [ ]:
constraints

In [ ]:
# For the objective function that will be maximized:
objective = plp.lpSum(x_vars[i,j,k] * dict_opt[i,j,k] 
                    for i in set_I 
                    for j in set_J
                    for k in set_K)

In [ ]:
objective

In [ ]:
# Adding the constraints to the model
opt_model.constraints = constraints

In [ ]:
# For maximization
opt_model.sense = plp.LpMaximize

In [ ]:
# Adding the objective to the model
opt_model.setObjective(objective)

In [ ]:
# solving with CBC
opt_model.solve()

In [ ]:
opt_model

In [ ]:
# The best way to see the result is to reflect them on a dataframe
opt_df_rre = pd.DataFrame.from_dict(x_vars, orient="index", columns = ["variable_object"])

# Arranging the index
opt_df_rre.index =  pd.MultiIndex.from_tuples(opt_df_rre.index, names=["column_i", "column_j", "column_k"])
opt_df_rre.reset_index(inplace=True)

# PuLP
opt_df_rre["solution_value"] = opt_df_rre["variable_object"].apply(lambda item: item.varValue)
opt_df_rre.drop(columns=["variable_object"], inplace=True)

In [ ]:
# Checking how many of the campaigns are suggested:
opt_df_rre.solution_value.value_counts()

In [ ]:
opt_df_rre.head()

In [ ]:
# Changing the column names
opt_df_rre = opt_df_rre.rename(columns = {'column_i':'day',
                         'column_j':'combination',
                         'column_k':'amount_spent_usd',
                         'solution_value':'solution'})

In [ ]:
# In order to understand which number corresponds to what combination, a dictionary created.
# This way it will be enabled to convert combination numbers into combination contents.
combin_dict_rvr = dict(zip(list(combin_dict.values()),list(combin_dict.keys())))
opt_df_rre['combination'] = opt_df_rre['combination'].apply(lambda x: combin_dict_rvr[x])

In [ ]:
opt_df_rre.combination[:8]

In [ ]:
# Split the combinations just like we combined them in the first iterations: 
opt_df_rre['age'] = opt_df_rre['combination'].str.split('_').str[0]
opt_df_rre['topic'] = opt_df_rre['combination'].str.split('_').str[1]
opt_df_rre['target_grouped'] = opt_df_rre['combination'].str.split('_').str[2]

In [ ]:
# Drop the combination column:
opt_df_rre.drop(columns=['combination'], inplace = True)

In [ ]:
# Multiply amount spent with 10 since it wasn't
opt_df_rre['amount_spent_usd'] = opt_df_rre['amount_spent_usd']*10

In [ ]:
opt_df_rre.head()

In [ ]:
# The suggested campaigns in the first day:
opt_df_rre[opt_df_rre['solution'] == 1][opt_df_rre['day'] == 1]

In [ ]:
# Let's convert the day column to date format yyyy-mm-dd:
import datetime

start = datetime.datetime.strptime("2019-08-01", "%Y-%m-%d") # need to be changed with the user input!
end = datetime.datetime.strptime("2019-08-08", "%Y-%m-%d") # need to be changed with the user input!
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

date_lst = []
for date in date_generated:
    date_lst.append(date.strftime("%Y-%m-%d"))
date_rng = list(range(1,8))

In [ ]:
# Creating the date column based on the day column
dict1 = dict(zip(date_rng, date_lst))
opt_df_rre['date'] = opt_df_rre['day'].apply(lambda x: dict1[x])

In [ ]:
# Converting to the datetime format
opt_df_rre['date'] = pd.to_datetime(opt_df_rre['date'], format = "%Y-%m-%d", errors='coerce')

In [ ]:
# Merging df_opt and opt_df to see which combinations are recommended
opt_df_rre_merged = df_opt_rre.merge(opt_df_rre[['amount_spent_usd','age','topic','target_grouped','date','solution']], 
                             on=['amount_spent_usd','age','topic','target_grouped','date'])

In [ ]:
# Excluding the unnecessary columns:
opt_df_rre_merged = opt_df_rre_merged[['date','age','topic','target_grouped','amount_spent_usd','final_prediction','solution']]

In [ ]:
len(opt_df_rre_merged)

In [ ]:
# Distribution of compaigns over labels:
opt_df_rre_merged[opt_df_rre_merged['solution'] == 1].final_prediction.value_counts()

In [ ]:
# How much did we spend each day?
opt_df_rre_merged[opt_df_rre_merged['solution'] == 1].groupby(['date']).sum()

Above chart proves that our optimization model works. The objective was to fit as much marketing campaigns as possible to the daily budget so that number of conversions can be maximized. The graph proves that it functions perfectly by filling the quota each day.

In [ ]:
# We also need to create a column indicating which day that date corresponds to:
opt_df_rre_merged['day_of_week'] = opt_df_rre_merged['date'].dt.dayofweek
days = {0:'Mon',1:'Tues',2:'Weds',3:'Thurs',4:'Fri',5:'Sat',6:'Sun'}
opt_df_rre_merged['day_of_week'] = opt_df_rre_merged['day_of_week'].apply(lambda x: days[x])

In [ ]:
# Rearranging the column order:
opt_df_rre_merged = opt_df_rre_merged[['date', 'day_of_week', 'age', 'topic', 'target_grouped', 'amount_spent_usd', 'final_prediction', 'solution']]

In [ ]:
# Dataframe that includes the weight for each label:
value_df = pd.DataFrame(list(labels.keys()), columns = ['index'])
value_df['weight'] = list(labels.values())
value_df

In [ ]:
# Dataframe that shows the amount of 0-2, 3-6, 7-12 and 12+ conversions
df1 = pd.DataFrame(opt_df_rre_merged[opt_df_ps_merged['solution'] == 1].final_prediction.value_counts()).reset_index().sort_values(['index'])
df1

In [ ]:
# Merging both:
evaluate_df = pd.merge(value_df, df1, how = 'left', on = ['index']).fillna(0)
evaluate_df

In [ ]:
'The expected amount of conversions for the designated period is: {}.'.format(int(sum(evaluate_df['weight'] * evaluate_df['final_prediction'])))

In [ ]:
opt_df_rre_merged.to_csv('/home/ec2-user/SageMaker/fb_budget_optimization_2019-07-17/optimized_aug_rre.csv')

In [ ]:
# Saving the file to S3:
from io import StringIO
import boto3

bucket='facebookconversionrates' # bucket name
data_key = 'modelling/optimized_aug_rre.csv' # name of the csv file that will be created
csv_buffer = StringIO()
opt_df_rre_merged.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, data_key).put(Body=csv_buffer.getvalue())

In [ ]:
opt_df_rre_merged.target_grouped.value_counts()

In [ ]:
opt_df